In [1]:
import json
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [14]:
science='science.json'
history='history.json'
geography='geography.json'
book='book.json'

In [15]:
science_questions=json.load(open(science,'r'))
history_questions=json.load(open(history,'r'))
geography_questions=json.load(open(geography,'r'))
book_questions=json.load(open(book,'r'))

In [20]:
X=[]
Y=[]
answer=[]
for i in science_questions:
    X.append(str(i[0]))
    Y.append(int(i[1]))
for i in geography_questions:
    X.append(str(i[0]))
    Y.append(int(i[1]))
for i in history_questions:
    X.append(str(i[0]))
    Y.append(int(i[1]))
for i in book_questions:
    answer.append(str(i['question']))

In [23]:
import re
for i in range(len(answer)):
  answer[i]=(re.sub(r'Q[0-9]+.','',answer[i]))

In [28]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer(oov_token='$')

#preparing vocabulary
tokenizer.fit_on_texts(answer)

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(X) 
ans_text_seq=tokenizer.texts_to_sequences(answer)
#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100,dtype='float32')
ans_text_seq = pad_sequences(ans_text_seq, maxlen=100,dtype='float32')
y_tr_seq=to_categorical(Y,3)
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

5490


In [25]:
X_train, X_test, y_train, y_test = train_test_split(x_tr_seq, y_tr_seq, test_size=0.33, random_state=42)

In [26]:
from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=100,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(3,activation='softmax')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          1647000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          219648    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 1,875,099
Trainable params: 1,875,099
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
history = model.fit(X_train,y_train,epochs=20,steps_per_epoch=300,validation_data=(X_test,y_test),verbose=1,callbacks=[es,mc])

Epoch 1/20
300/300 [==============================] - ETA: 0s - loss: 0.9797 - acc: 0.5517
Epoch 00001: val_acc improved from -inf to 0.58491, saving model to best_model.h5
300/300 [==============================] - 29s 97ms/step - loss: 0.9797 - acc: 0.5517 - val_loss: 0.9554 - val_acc: 0.5849
Epoch 2/20
300/300 [==============================] - ETA: 0s - loss: 0.9754 - acc: 0.5483
Epoch 00002: val_acc did not improve from 0.58491
300/300 [==============================] - 29s 96ms/step - loss: 0.9754 - acc: 0.5483 - val_loss: 0.9563 - val_acc: 0.5849
Epoch 3/20
300/300 [==============================] - ETA: 0s - loss: 0.9666 - acc: 0.5483
Epoch 00003: val_acc did not improve from 0.58491
300/300 [==============================] - 29s 95ms/step - loss: 0.9666 - acc: 0.5483 - val_loss: 0.9668 - val_acc: 0.5849
Epoch 4/20
300/300 [==============================] - ETA: 0s - loss: 0.9490 - acc: 0.5650
Epoch 00004: val_acc did not improve from 0.58491
300/300 [==========================

In [29]:
res=model.predict(ans_text_seq)

In [48]:
def pre(result):
  mapper={0:'science',1:'history',2:'geography'}
  result=list(result)
  return mapper[result.index(max(result))]

In [49]:
pre(res[2])

'geography'

In [50]:
for i in range(len(res)):
  book_questions[i]['class']=pre(res[i])

In [51]:
with open("subject.json",'w') as file: 
  json.dump(book_questions,file,indent=4)